In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob
import time
from IPython import display
import ipywidgets as widgets
from IPython.display import display as display_widget

In [2]:
#CAMERA CALIBRATION

# Chessboard dimensions
chessboard_size = (9, 6)

# Criterio de terminación de subpixeles
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# 3D points
objp = np.zeros((np.prod(chessboard_size), 3), np.float32)
objp[:, :2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2)

objpoints = [] # 3D points
imgpoints = [] # 2D points

# Load data (images)
images = glob.glob("../assets/chessboard/*.jpeg")

#Find Corners
for i, fname in enumerate(images):
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    ret, corners = cv2.findChessboardCorners(gray, chessboard_size, None)
    if ret:
        #Refine corner positions
        ref_corners = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        
        #Save points
        objpoints.append(objp)
        imgpoints.append(ref_corners)

        #Save Image with detected corner positions
        c_img = img.copy()
        cv2.drawChessboardCorners(c_img, chessboard_size, ref_corners, ret)
        cv2.imwrite(f'../res/detected_corners/corners_{i}.jpg', c_img)

# Calibration

#Instrinsics: Camera Matrix, Distortion Coefficients - Extrinscics: rvecs (Rotation), tvecs (Translation)
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

#Camera Matrix with Intrinsic Parameters : Focal Lenght (f_x, f_y), Optical Centers (c_x, c_y)
print("Camera Matrix:\n", mtx)
#Distortion Coefficients
print("Distortion Coefficients:\n", dist)

# Data set image example correction
img = cv2.imread(images[3])
h, w = img.shape[:2]

#Optimized Camera Matrix
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

#Undistort
dst = cv2.undistort(img, mtx, dist, None, newcameramtx)

#Crop Image by Region of Interest (ROI)
x, y, w, h = roi
dst = dst[y: y + h, x: x + w]

cv2.imwrite("../res/example_img.jpeg", img)
cv2.imwrite("../res/example_img_correction.jpeg", dst)


Camera Matrix:
 [[1.35682688e+03 0.00000000e+00 5.88743980e+02]
 [0.00000000e+00 1.36704013e+03 7.73835229e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Distortion Coefficients:
 [[ 0.23982088 -1.64384751 -0.0068643  -0.01283784  1.95547747]]


True

In [3]:
##CORRECTION ALL DATA SET IMAGES
for i, _ in enumerate(images):
    img = cv2.imread(images[i])
    h, w = img.shape[:2]

    #Optimized Camera Matrix
    newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

    #Undistord
    dst = cv2.undistort(img, mtx, dist, None, newcameramtx)

    #Crop Image by Region of Interest (ROI)
    x, y, w, h = roi
    dst = dst[y: y + h, x: x + w]

    cv2.imwrite(f"../res/corrected/correction_{i}.jpeg", dst)

In [4]:
##REPROJECTION ERROR

mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2)/len(imgpoints2)
    mean_error += error

print(f"Mean Error: {mean_error}")
print(f"Total Error: {mean_error / len(objpoints)}")

Mean Error: 3.011222556066742
Total Error: 0.21508732543333872


In [ ]:
##REAL TIME VIDEO CORRECTION

########################################################################################
#Jupyter Notebook interactive crap buttons for video feed.

# Last frame captured, counter GLOBALS
frame_c = 0
cap = None

#Master output
window = widgets.Output()

#Screenshot Button
ss_button = widgets.Button(description = "Screenshot")
button_output = widgets.Output()

#Callback
def screenshot(b):
    global frame_c, cap
    if cap is not None:
        #Capture the current frame
        ret, frame = cap.read() 
        if ret:
            h, w = frame.shape[:2]
            newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))
            corrected = cv2.undistort(frame, mtx, dist, None, newcameramtx)
            s2sview = np.hstack((frame, corrected))
            
            #Screenshot
            cv2.imwrite(f"../res/real_time_corrected/real_time_correction_{frame_c}.jpeg", s2sview)
            with button_output:
                print(f"Screenshot saved: {output_dir}real_time_correction_{frame_c}.jpeg")

            frame_c += 1

#Link the button's click event to the function
ss_button.on_click(screenshot)

#Vstacked. UP: Button. Bottom: Video feed

display_widget(widgets.VBox([button_output, window]))

#Display Button above video feed
with button_output:
    display_widget(ss_button)

##############################################################################################
with window:
    #MAIN LOGIC
    #Main WebCam binding (usual)
    #cap = cv2.VideoCapture(0)

    #iF USB Camera, Index is not necessarily 0
    for i in range(10):
        cap = cv2.VideoCapture(i)
        if cap.isOpened():
            with button_output:
                print(f"Camera index: {i}")
            break

    while True:
        global ss_req, frame_c, ss_frame
        ret, frame = cap.read()
        if not ret:
            with button_output:
                print("Shit happens.")
                break
        
        h, w = frame.shape[:2]
        
        #Optimized Camera Matrix
        newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))
        
        #Undistorted frame -> corrected
        corrected = cv2.undistort(frame, mtx, dist, None, newcameramtx)
        
        # Side to side video feed. (Left Original, Right Corrected)
        s2sview = np.hstack((frame, corrected))
        
        #Captions
        cv2.putText(s2sview, "LIVE FEED", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
        cv2.putText(s2sview, "CORRECTED", (w + 10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
        cv2.putText(s2sview, "Stop cell execution to quit", (10, h - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                    

        # s2sview frame to RGB for displaying
        s2sview_rgb = cv2.cvtColor(s2sview, cv2.COLOR_BGR2RGB)



        #LIVE FEED
        display.clear_output(wait = True)
        plt.figure(figsize = (10, 5))
        plt.imshow(s2sview_rgb)
        plt.axis('off')
        plt.show()

        frame_c += 1
    
    cap.release()
    cv2.destroyAllWindows()